In [ ]:
import pygui as pgui
pgui.create_interface()


# Example usage:

        # F =>  String: Faction                  'Alloyn'              F=Alloyn
        # D =>  String: Deckname                 'Insane'              D~Forge
        # FB => String: Forgeborn               'Ironbeard'            FB=Ironbeard
        # C  => List:   Cardnames               'Digitize'             C~Digitize
        # A  => List:   Forgeborn - Ability     'Army Commander'       A~'Army Commander'
        # K  => Dict:   Composition             'Robot'                K:Robot > 3

In [1]:
import pygui as pgui
import pandas as pd
import qgrid
from IPython.display import display
from MongoDB.DatabaseManager import DatabaseManager
import GlobalVariables

df = pgui.collection_to_df('Deck')

GlobalVariables.username = 'tmind'
myDB = DatabaseManager(GlobalVariables.username)
db_name = myDB.get_current_db_name()

# Get all Decks from the database
deck_cursor = myDB.find('Deck', {})
df_decks = pd.DataFrame(list(deck_cursor))

# Create a DataFrame from the 'stats' sub-dictionary
df_decks_stats = pd.DataFrame()
for deck in myDB.find('Deck', {}):
    if 'stats' in deck:


        stats = deck.get('stats', {})
        # Convert the 'card_types' sub-dictionary into a DataFrame
        card_types_df = pd.DataFrame.from_dict(stats['card_types'], orient='index')

        # Convert the 'creature_averages' sub-dictionary into a DataFrame
        creature_averages_df = pd.DataFrame.from_dict({k: v for k, v in stats['creature_averages'].items()}, orient='index')

                
        # Combine the two DataFrame
        deck_stats_df = pd.concat([creature_averages_df, card_types_df], axis=1)
        deck_stats_df = deck_stats_df.fillna('')
        display(deck_stats_df)
        
        

qgrid_df_DeckStats = qgrid.show_grid(df_decks_stats, show_toolbar=True)            
display(qgrid_df_DeckStats)

df_filtered = df[[ 'registeredDate', 'name', 'cardSetName', 'faction', 'forgebornId']]            
# Merge the filtered dataframe with the deck stats dataframe
merged_df = df_filtered.merge(df_decks_stats, left_index=True, right_index=True)
qgrid_df = qgrid.show_grid(df_filtered, show_toolbar=True)            
display(qgrid_df)

,1,2,3,count,Metamind,Scientist,Robot,Warrior,Mage,Exalt
attack,3.285714,6.142857,8.714286,NaN,NaN,NaN,NaN,NaN,NaN,NaN
health,4.714286,5.857143,8.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Creature,NaN,NaN,NaN,7.0,1.0,1.0,4.0,3.0,1.0,
Spell,NaN,NaN,NaN,3.0,,,,,,1.0
